In [3]:
import torch
from torchvision import models, transforms
from PIL import Image
import cv2
import numpy as np
import os

# Função para carregar e pré-processar uma imagem para ResNet18
def preprocess_image_resnet18(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)

# Função para carregar e pré-processar uma imagem para YOLOv10
def preprocess_image_yolo(image_path):
    image = cv2.imread(image_path)
    resized = cv2.resize(image, (640, 640))  # Tamanho usado no treinamento do YOLO
    input_tensor = torch.from_numpy(resized).permute(2, 0, 1).float() / 255.0
    return input_tensor.unsqueeze(0)

# Função para realizar inferência com o modelo ResNet18
def predict_resnet18(model, image_tensor):
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

# Função para realizar inferência com o modelo YOLOv10
def predict_yolo(model, image_tensor):
    with torch.no_grad():
        outputs = model(image_tensor)
    return outputs  # Saída do modelo YOLO (caixas delimitadoras, scores, classes)

# Função para comparar resultados
def compare_results(image_path, resnet18_model, yolo_model):
    # Pré-processar imagens
    resnet18_input = preprocess_image_resnet18(image_path)
    yolo_input = preprocess_image_yolo(image_path)
    
    # Inferências
    resnet18_prediction = predict_resnet18(resnet18_model, resnet18_input)
    yolo_prediction = predict_yolo(yolo_model, yolo_input)
    
    print(f"Image: {image_path}")
    print(f"ResNet18 Prediction: {resnet18_prediction}")
    print(f"YOLOv10 Prediction: {yolo_prediction}")
    print("-" * 50)

# Carregar os modelos
# Substitua pelos seus arquivos de modelo treinados
resnet18_model_path = "./modelos/resnet18_model.pth"
yolo_model_path = "./modelos/best.pt"

resnet18_model = models.resnet18()
resnet18_model.fc = torch.nn.Linear(resnet18_model.fc.in_features, num_classes)  # Ajustar número de classes
resnet18_model.load_state_dict(torch.load(resnet18_model_path))
resnet18_model.eval()

yolo_model = torch.load(yolo_model_path)
yolo_model.eval()

# Pasta de imagens para comparar
image_folder = "./images"  # Substitua pela sua pasta
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Comparar resultados
for image_file in image_files:
    compare_results(image_file, resnet18_model, yolo_model)


C:\Users\Caio Fuzita\AppData\Local\Temp\ipykernel_12008\1167955147.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet18_model.load_state_dict(torch.load(resnet18_mo

FileNotFoundError: [Errno 2] No such file or directory: './modelos/yolov10_model.pth'